<a href="https://colab.research.google.com/github/shashanksn111/AML/blob/main/AIML5152_Lab_FinalExamSet2_Shashank__S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Reg* Num:  _231057022_

---
$$\color{orange}{AML\ 5152\,\lvert\, Applied\ Machine\ Learning\,\lvert\,Lab\ Final\,\lvert\,Odd\ Semester\ 2023}$$
---

**Instructions:**
1. Fill the reg num at the top of this notebook
2. When a code template is provided, you have to fill the code template. You should not replace the code template with different code from elsewhere
3. Upload your Jupyter notebook with all its outputs intact here: https://tinyurl.com/tckf29w4
4. Do not solicit inputs from others. Plagiarism check will be performed after the exam
5. You will be orally asked why you did each design choice along the way. If you cannot defend your choice, then some marks will be deducted

### **Problem Statement**

**PS: DO NOT FORGET TO TRAIN-TEST SPLIT AT AN APPROPRIATE TIME IN THE ENTIRE FLOW**.

You decide where you want to position the train test split in the stages below

#### I. Create a pandas dataset for apples satisfying the following constraints:
1. Datset should have six columns - weight, volume, pesticide per apple, discoloration, machine_plucked and apple_type and 100 records with 70 Gala apples, 15 Fuji apples and 15 Red apples
2. Apple type is a target variable. Remaining are predictor variables.
3. There are 3 apple types: Gala, Fuji, Red.
4. Machine Plucked is either yes or no.
5. Simulate the data randomly such that subsequent simulations produce very similar or identical data
6. Simulate the data for the 3 types of apples according to the following rules:
	* Gala apple weights are normally distributed with mean of 155 g and a standard deviation of 5 g.
	* Fuji apple weights are uniformly distributed between 200 and 250 gram
	* Red apple weights are distributed according to triangular distribution with minimum weight of 100g, maximum weight of 190 g and the most frequent weight being 170g
7. Apple volumes are normally distributed for the three apple types Gala, Fuji and Red with a mean of 187cc, 270cc and 150cc respectively and variance of 25cc respectively
8. A pesticide Quinalphos was dissolved in water and sprayed at the rate of 500 gm per 100 apples. The spray was unequal and had a variance of 4 milligram per apple. This pestiside dosage is common to all three apple types. This data will be used to populate "pesticide per apple" feature
9. Discoloration of the apple is equal to the percentile of the pesticide per apple

#### II. Introduce NaNs
1. Randomly introduce NaN for weight and volume feature for 25% of the records such that the **fraction of NaN for each apple type is proportional to the ratio of samples**.
2. Pesticide_per_apple data should be randomly nulled out for data beyond 75th percentile
3. Randomly introduce NaN for machine plucked apples for 5% of records
4. Randomly introduce NaN for apple type for 10% of records

#### III Transform, Train/Test Split and Impute
Ask yourself these questions and do accordingly:
1. Will you do train test split before or after doing train test split?
2. Will you do transformation after imputation or before?
3. Will you do split before transformation?

According to your choice do these three in the order you deem fit

1. Impute the data for relevant columns using an appropriate imputation method fit for each scenario
2. If there are any records that you feel should be deleted, then please do so
3. Do a train test split 80:20 such that the fraction of NaN for each apple type is proportional to the ratio of samples of that apple type
4. Do any other data transformation you feel is needed

#### IV Feature Elimination and Feature Selection
1. If there are any features that you can immediately drop without any exploration, programming then please do so first
2. Check which features have highest predictive power wrt target variable
3. Check features on which target is dependent. Use a mechanism that is different from previous method for this.
4. Base on the above two checks, choose 2 features for predicting apple type

#### V ML Prediction
1. Apply Logistic Regression to predict apple type
2. Choose a  metric that you think is most suitable for this scenario

In [97]:
from math import sqrt

import numpy as np
import pandas as pd

np.random.seed(42)

---

#### I. Create a pandas dataset for apples satisfying the following constraints:
1. Datset should have six columns - weight, volume, pesticide per apple, discoloration, machine_plucked and apple_type and 200 records with 70 Gala apples, 15 Fuji apples and 15 Red apples
2. Apple type is a target variable. Remaining are predictor variables.
3. There are 3 apple types: Gala, Fuji, Red.

In [98]:
total_records = 100
apple_types = ["Gala", "Fuji", "Red"]
apple_ratios = (0.7, 0.15, 0.15)

In [99]:
# Define the number of records and apple types
total_records = 100
apple_types = ["Gala", "Fuji", "Red"]

# Define the number of apples for each type
apple_ratios = [0.7, 0.15, 0.15]
apple_counts = [int(ratio * total_records) for ratio in apple_ratios]

# Define the column names
columns = ["weight", "volume", "pesticide_per_apple", "discoloration", "machine_plucked", "apple_type"]

# Create an empty dataframe
df = pd.DataFrame(columns=columns)

# Simulate the data for each apple type
for i, apple_type in enumerate(apple_types):
    # Generate the weight data
    if apple_type == "Gala":
        weight_mean = 155
        weight_std = 5
        weights = np.random.normal(weight_mean, weight_std, apple_counts[i])
    elif apple_type == "Fuji":
        weight_low = 200
        weight_high = 250
        weights = np.random.uniform(weight_low, weight_high, apple_counts[i])
    elif apple_type == "Red":
        weight_min = 100
        weight_max = 190
        weight_mode = 170
        weights = np.random.triangular(weight_min, weight_mode, weight_max, apple_counts[i])

    # Generate the volume data
    if apple_type == "Gala":
        volume_mean = 187
        volume_std = 5
        volumes = np.random.normal(volume_mean, volume_std, apple_counts[i])
    elif apple_type == "Fuji":
        volume_mean = 270
        volume_std = 5
        volumes = np.random.normal(volume_mean, volume_std, apple_counts[i])
    elif apple_type == "Red":
        volume_mean = 150
        volume_std = 5
        volumes = np.random.normal(volume_mean, volume_std, apple_counts[i])

    # Generate the pesticide per apple data
    pesticide_mean = 500 / total_records
    pesticide_std = 4
    pesticide_per_apple = np.random.normal(pesticide_mean, pesticide_std, apple_counts[i])

    # Generate the discoloration data
    discoloration = [np.percentile(pesticide_per_apple, np.searchsorted(pesticide_per_apple, p)) for p in pesticide_per_apple]

    # Generate the machine plucked data
    machine_plucked = np.random.choice(["Yes", "No"], apple_counts[i])

    # Generate the apple type data
    apple_type_data = [apple_type] * apple_counts[i]

    # Create a dataframe for the current apple type
    apple_type_df = pd.DataFrame({
        "weight": weights,
        "volume": volumes,
        "pesticide_per_apple": pesticide_per_apple,
        "discoloration": discoloration,
        "machine_plucked": machine_plucked,
        "apple_type": apple_type_data
    })

    # Append the dataframe to the main dataframe
    df = df.append(apple_type_df, ignore_index=True)

# Display the final dataframe
df


<ipython-input-99-a6ff9e117983>:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(apple_type_df, ignore_index=True)
<ipython-input-99-a6ff9e117983>:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(apple_type_df, ignore_index=True)
<ipython-input-99-a6ff9e117983>:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(apple_type_df, ignore_index=True)


,weight,volume,pesticide_per_apple,discoloration,machine_plucked,apple_type
0,157.483571,188.806978,5.909840,-0.336751,Yes,Gala
1,154.308678,194.690183,10.228571,7.058192,No,Gala
2,158.238443,186.820870,-1.429933,-1.429933,Yes,Gala
3,162.615149,194.823218,5.738535,-0.494658,No,Gala
4,153.829233,173.901274,6.039531,-0.336751,No,Gala
...,...,...,...,...,...,...
95,146.324235,150.902463,6.070544,-4.162388,No,Red
96,140.526233,155.355208,-0.369251,-6.423577,Yes,Red
97,155.902021,153.822391,-8.456382,-8.456382,No,Red
98,166.069730,144.444181,9.494167,-4.162388,Yes,Red


4. Machine Plucked is either yes or no.

In [100]:
machine_plucked = np.random.choice(['Yes', 'No'], total_records, p=[0.5, 0.5])
machine_plucked

array(['Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No',
       'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No',
       'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes',
       'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No',
       'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No',
       'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes',
       'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes',
       'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes',
       'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes'],
      dtype='<U3')

**Simulate Gala apples**

1. Gala apple weights are normally distributed with mean of 155 g and a standard deviation of 5 g.
2. Gala apple volumes are normally distributed with mean of 187 cc and variance of 25 cc $^2$

**Note: Check if the np.random functions accept standard deviation or variance as arguments and accordingly adjust**  

In [101]:
from enum import Enum
class Gala(Enum):
    apple_ratio = 0.7

    volume_avg = 187
    volume_variance = 25

    weight_mean = 155
    weight_standard_dev = 5

In [102]:
gala_weights = np.random.normal(Gala.weight_mean.value,
                                Gala.weight_standard_dev.value,
                                int(Gala.apple_ratio.value * total_records))

gala_volumes = np.random.normal(size=int(Gala.apple_ratio.value * total_records),
                                scale=sqrt(Gala.volume_variance.value),
                                loc=Gala.volume_avg.value)


**Simulate Fuji apples**

1. Fuji apple weights are uniformly distributed between 200 and 250 gram
2. Fuji apple volumes are normally distributed with a mean os 270 cc and variance of 25 cc $^2$

In [103]:
fuji_dict = {
    "apple_ratio": 0.15,
    "volume_mean": 270,
    "volume_variance": 25,
    "weight_high": 250,
    "weight_low": 200
}

In [104]:
fuji_weights = np.random.uniform(size=int(fuji_dict["apple_ratio"] * total_records),
                                 high=fuji_dict["weight_high"],
                                 low=fuji_dict["weight_low"])

fuji_volumes = np.random.normal(size=int(fuji_dict["apple_ratio"] * total_records),
                                scale=sqrt(fuji_dict["volume_variance"]),
                                loc=fuji_dict["volume_mean"])


**Simulate Red Apples**


1. Red apple weights are distributed according to triangular distribution with minimum weight of 100g, maximum weight of 190 g and the most frequent weight being 170g
2. Red apple volumes are normally distributed with mean of 150cc and standard deviation of 5cc

In [105]:
red_weights = np.random.triangular(100, 170, 190, 15)
red_volumes = np.random.normal(size=int(0.15*total_records), scale=150, loc=5)


8. A pesticide Quinalphos was dissolved in water and sprayed at the rate of 500 gm per 100 apples. The spray was unequal and had a variance of 4 milligram per apple. This pestiside dosage is common to all three apple types. This data will be used to populate "pesticide per apple" feature

In [106]:
#pesticide_per_apple = np.random.normal(50/total_records, 3, total_records)

from scipy import stats
pesticide_per_apple_distribution = stats.norm(
    loc=500/total_records,
    scale=sqrt(4)
)
pesticide_per_apple = pesticide_per_apple_distribution.rvs(size=total_records)
print(pesticide_per_apple)


[ 7.07507989  3.9799672   4.46025013  3.04247257  4.11141348  5.75460099
  6.51397723  3.15566935  6.73921184  7.71127572  5.82686981  8.75359163
  3.4524216   2.51069059  1.4425595   7.99208862  6.30873131  4.88883066
  5.55993725  2.74902191  9.89150396  5.25844236  5.21878959  6.45153325
  5.96201846  5.44776805  3.41905109  5.94293671  8.76404899  7.69084009
  8.18637325  3.97756865  3.02079036  4.74842616  5.11144982  7.18838304
  1.61507074  8.05910064  4.6839842   4.14623786  2.97579125  1.69028666
  6.64634117  5.14663593  2.4200782   2.40984246  4.3284306   8.33804305
  4.4808173   1.99371409  4.50851387  4.45455286 -0.39377329  4.89141027
  4.53813094  6.39241273  8.69791219  7.25313006  4.46222262  2.78694818
 10.14671961  5.11843687  5.02785858  4.95174983  5.39616952  4.71127918
  3.85267599  3.90628212  4.93449346  3.91315046  3.57430843  5.21286046
  4.49004557  8.00798598 -0.30193962  7.1830137   7.49217038  0.85321954
  4.31462481  4.25711827  2.18497661  3.44436662  2

In [107]:
discoloration = [pesticide_per_apple_distribution.cdf(rec) for rec in pesticide_per_apple]
print(discoloration)

[0.8502578550095234, 0.3050199865359049, 0.3936282353369686, 0.1638483708419463, 0.3284152867509085, 0.6470248470899724, 0.7754716588356634, 0.17822117367068724, 0.8077420993014939, 0.9123927900142659, 0.6603559965473047, 0.9697269576923726, 0.2195277303818261, 0.10662935150065944, 0.03764281886938667, 0.9326789458301921, 0.7435618724073488, 0.47783633823261357, 0.610249212361812, 0.13019092793860576, 0.9927724799854053, 0.551408682032652, 0.54355531886255, 0.7660090745259777, 0.6847450312375958, 0.588576226500906, 0.2146253673466299, 0.681346844428252, 0.9700836551522274, 0.9107551873853961, 0.9444408347871412, 0.3046000214040668, 0.16118365651848593, 0.44995029788946395, 0.5222195234010824, 0.8630645047353573, 0.045278709289926, 0.9369359626703674, 0.43722528736986266, 0.3347329770660814, 0.15574407532738477, 0.04897678751076649, 0.794794502830757, 0.5292234527853265, 0.09853211711374926, 0.09764652851680644, 0.3685166145693303, 0.9524434430131289, 0.39758950414992006, 0.066401092245

In [108]:
# Convert the discoloration into percentage and only integer precision

df['discoloration'] = (df['discoloration'] * 100).astype(int)
print(discoloration_percentage)

[11, 36, 14, 90, 52, 56, 41, 57, 99, 38, 25, 70, 0, 52, 35, 16, 69, 52, 3, 25, 0, 61, 61, 21, 70, 9, 16, 66, 24, 47, 28, 34, 35, 23, 61, 26, 50, 23, 6, 81, 69, 98, 14, 41, 44, 83, 30, 12, 88, 37, 81, 77, 48, 43, 34, 49, 80, 21, 54, 13, 16, 48, 2, 98, 11, 48, 87, 19, 1, 32, 19, 1, 61, 58, 87, 13, 5, 34, 46, 82, 92, 88, 60, 78, 26, 56, 11, 52, 53, 95, 40, 53, 59, 32, 21, 90, 16, 9, 75, 87, 32, 33, 61, 23, 21, 81, 72, 90, 10, 30, 61, 17, 73, 88, 27, 51, 92, 96, 77, 72, 19, 64, 71, 41, 27, 10, 4, 2, 71, 55, 31, 37, 58, 43, 9, 81, 61, 60, 79, 20, 88, 44, 81, 76, 14, 69, 62, 51, 46, 46, 79, 71, 51, 6, 68, 76, 98, 85, 2, 53, 96, 73, 62, 81, 26, 68, 40, 49, 79, 17, 90, 33, 60, 8, 76, 92, 30, 13, 60, 92, 7, 13, 32, 20, 20, 1, 72, 63, 4, 92, 95, 22, 87, 28, 57, 20, 98, 38, 55, 24]


**Concatenate all features and target variables and make the dataframe**

1. Be sure to line up the simulated records per the apple type and concatenate
2. Display the final dataframe

In [109]:
# Concatenate all features and target variables and make the dataframe
df_final = pd.concat([df['weight'], df['volume'], df['pesticide_per_apple'], df['discoloration'], df['machine_plucked'], df['apple_type']], axis=1)

# Display the final dataframe
df_final

,weight,volume,pesticide_per_apple,discoloration,machine_plucked,apple_type
0,157.483571,188.806978,5.909840,-33,Yes,Gala
1,154.308678,194.690183,10.228571,705,No,Gala
2,158.238443,186.820870,-1.429933,-142,Yes,Gala
3,162.615149,194.823218,5.738535,-49,No,Gala
4,153.829233,173.901274,6.039531,-33,No,Gala
...,...,...,...,...,...,...
95,146.324235,150.902463,6.070544,-416,No,Red
96,140.526233,155.355208,-0.369251,-642,Yes,Red
97,155.902021,153.822391,-8.456382,-845,No,Red
98,166.069730,144.444181,9.494167,-416,Yes,Red


---

#### II. Introduce NaNs
1. Randomly introduce NaN for weight and volume feature for 25% of the records such that the **fraction of NaN for each apple type is proportional to the ratio of samples**.
2. Pesticide_per_apple data should be randomly nulled out for data beyond 75th percentile. Use the discoloration data (which is nothing but the percentile) to make this determination
3. Randomly introduce NaN for machine plucked apples for 5% of records
4. Randomly introduce NaN for apple type for 10% of records

In [110]:
import numpy as np

# Calculate the number of NaN values for each column
nan_count_weight_volume = int(0.25 * len(df))
nan_count_pesticide = int(0.75 * len(df[df['discoloration'] <= 75]))
nan_count_machine_plucked = int(0.05 * len(df))
nan_count_apple_type = int(0.1 * len(df))


# Randomly select indices to set as NaN for weight and volume columns
nan_indices_weight_volume = np.random.choice(len(df), size=min(nan_count_weight_volume, len(df)), replace=False)
df.loc[nan_indices_weight_volume, ['weight', 'volume']] = np.nan

# Randomly select indices to set as NaN for pesticide_per_apple column
nan_indices_pesticide = np.random.choice(df[df['discoloration'] > 75].index, size=min(nan_count_pesticide, len(df[df['discoloration'] > 75])), replace=True)
df.loc[nan_indices_pesticide, 'pesticide_per_apple'] = np.nan

# Randomly select indices to set as NaN for machine_plucked column
nan_indices_machine_plucked = np.random.choice(len(df), size=min(nan_count_machine_plucked, len(df)), replace=True)
df.loc[nan_indices_machine_plucked, 'machine_plucked'] = np.nan

# Randomly select indices to set as NaN for apple_type column
nan_indices_apple_type = np.random.choice(len(df), size=min(nan_count_apple_type, len(df)), replace=False)
df.loc[nan_indices_apple_type, 'apple_type'] = np.nan

# Display the modified dataframe
df

,weight,volume,pesticide_per_apple,discoloration,machine_plucked,apple_type
0,157.483571,188.806978,5.909840,-33,Yes,Gala
1,154.308678,194.690183,NaN,705,No,Gala
2,NaN,NaN,-1.429933,-142,Yes,Gala
3,162.615149,194.823218,5.738535,-49,No,Gala
4,153.829233,173.901274,6.039531,-33,No,Gala
...,...,...,...,...,...,...
95,146.324235,150.902463,6.070544,-416,No,Red
96,140.526233,155.355208,-0.369251,-642,Yes,Red
97,155.902021,153.822391,-8.456382,-845,No,Red
98,166.069730,144.444181,9.494167,-416,Yes,Red


---

#### III Transform, Train/Test Split and Impute
Ask yourself these questions and do accordingly:
1. Will you do train test split before or after doing train test split?
2. Will you do transformation after imputation or before?
3. Will you do split before transformation?

According to your choice do these three in the order you deem fit

1. Impute the data for relevant columns using an appropriate imputation method fit for each scenario
2. If there are any records that you feel should be deleted, then please do so
3. Do a train test split 80:20 such that the fraction of NaN for each apple type is proportional to the ratio of samples of that apple type
4. Do any other data transformation you feel is needed

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Step 1: Impute the data
numeric_columns = ['weight', 'volume', 'pesticide_per_apple', 'discoloration']
imputer = SimpleImputer(strategy='mean')
imputed_data = imputer.fit_transform(df[numeric_columns])

# Replace the imputed values in the original dataframe
df[numeric_columns] = imputed_data

In [112]:
# Step 2: Delete records if needed
cleaned_data = df.dropna()

In [113]:
# Step 3: Train/test split
# Calculate the number of samples for each apple type
apple_counts = cleaned_data['apple_type'].value_counts()
gala_samples = int(apple_counts['Gala'] * 0.8)
fuji_samples = int(apple_counts['Fuji'] * 0.8)
red_samples = int(apple_counts['Red'] * 0.8)

In [114]:
# Split the data into train and test sets
train_data = cleaned_data[cleaned_data['apple_type'] == 'Gala'].sample(gala_samples).append(
    cleaned_data[cleaned_data['apple_type'] == 'Fuji'].sample(fuji_samples)).append(
    cleaned_data[cleaned_data['apple_type'] == 'Red'].sample(red_samples))

test_data = cleaned_data.drop(train_data.index)

<ipython-input-114-7d2af4cf470f>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = cleaned_data[cleaned_data['apple_type'] == 'Gala'].sample(gala_samples).append(
<ipython-input-114-7d2af4cf470f>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_data[cleaned_data['apple_type'] == 'Fuji'].sample(fuji_samples)).append(


In [115]:
# Step 4: Perform any other data transformation if needed
train_data.head()
test_data.head()

,weight,volume,pesticide_per_apple,discoloration,machine_plucked,apple_type
10,152.682912,185.901641,6.001971,-33.0,Yes,Gala
16,149.935844,191.577011,4.088956,705.0,Yes,Gala
32,154.932514,185.286427,4.691593,-49.0,No,Gala
33,149.711445,182.988614,6.364608,14.0,Yes,Gala
44,147.607390,186.038195,4.106149,-49.0,Yes,Gala


In [116]:
from sklearn.preprocessing import StandardScaler
# Standard scaling on numerical features
scaler = StandardScaler()
train_data[numeric_columns] = scaler.fit_transform(train_data[numeric_columns])
test_data[numeric_columns] = scaler.transform(test_data[numeric_columns])

# Display the transformed and split data
print("Train Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())

Train Data:
      weight    volume  pesticide_per_apple  discoloration machine_plucked  \
56 -0.059712 -0.011205            -0.690273      -0.152332              No   
40 -0.276216 -0.606166             1.012406       0.489851             Yes   
61 -0.491954 -0.249561             0.939221       0.489851              No   
37 -0.906099 -0.230538             1.946204       2.099786             Yes   
34 -0.059712 -0.011205             0.618776       0.035843              No   

   apple_type  
56       Gala  
40       Gala  
61       Gala  
37       Gala  
34       Gala  

Test Data:
      weight    volume  pesticide_per_apple  discoloration machine_plucked  \
10 -0.556797 -0.301281             0.589225      -0.104541             Yes   
16 -0.685058 -0.097605             0.049776       2.099786             Yes   
32 -0.451763 -0.323360             0.219713      -0.152332              No   
33 -0.695536 -0.405823             0.691485       0.035843             Yes   
44 -0.793774 -0.29638

---

#### IV Feature Elimination and Feature Selection
1. If there are any features that you can immediately drop without any exploration, programming then please do so first
2. Check which features have highest predictive power wrt target variable
3. Check features on which target is dependent. Use a mechanism that is different from previous method for this.
4. Base on the above two checks, choose 2 features for predicting apple type


In [117]:
# Drop irrelevant features
df = df.drop(['weight', 'volume'], axis=1)

In [118]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2_contingency

# Assuming 'X' is your feature set and 'y' is your target variable
# Replace 'X' and 'y' with your actual feature set and target variable
X = df[['discoloration' ,'machine_plucked', 'pesticide_per_apple' ]]
y = df['apple_type']


# Step 1: Encode categorical variables
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

# Step 2: Check feature importance using a machine learning model
# Train a random forest classifier
rf = RandomForestClassifier()
rf.fit(X_encoded, y)

# Get the feature importances
feature_importances = rf.feature_importances_

# Step 3: Check features on which target is dependent using chi-square test
dependent_features = []
for feature in X.columns:
    contingency_table = pd.crosstab(train_data[feature], train_data['apple_type'])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    if p_value < 0.05:
        dependent_features.append(feature)

# Step 4: Choose 2 features for predicting apple type
selected_features = dependent_features[:2]

# Print the selected features
print(selected_features)
# Perform chi-square test for each feature
dependent_features = []
for feature in X.columns:
    contingency_table = pd.crosstab(train_data[feature], train_data['apple_type'])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    if p_value < 0.05:
        dependent_features.append(feature)

# Step 4: Choose 2 features for predicting apple type
selected_features = dependent_features[:2]

# Print the selected features
print(selected_features)


ValueError: ignored

---

#### V ML Prediction
1. Apply Logistic Regression to predict apple type
2. Choose a  metric that you think is most suitable for this scenario

In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Encode the categorical variables
label_encoder = LabelEncoder()
X_encoded = X.apply(label_encoder.fit_transform)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create the Logistic Regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict the apple types for the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)
# Print the accuracy
print("Accuracy:", accuracy)
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict the apple types for the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)


ValueError: ignored